In [ ]:
!pip install pretty_html_table

In [ ]:
import pandas as pd
import os
import re
import requests
import itertools
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from collections import  Counter
from spacy.lang.es import stop_words
import gensim
from gensim.matutils import corpus2dense
from gensim.models import CoherenceModel
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pretty_html_table import build_table
import IPython
import spacy
spacy.cli.download('es_core_news_lg')
from sklearn import linear_model
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

%matplotlib inline


✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')


In [ ]:




#Aplicaré la misma función de preprocesamiento aplicada al conjunto train

def quitar_acentos(old):

    new = old.lower()

    new = re.sub(r'[àáâãäå]', 'a', new)
    new = re.sub(r'[èéêë]', 'e', new)
    new = re.sub(r'[ìíîï]', 'i', new)
    new = re.sub(r'[òóôõö]', 'o', new)
    new = re.sub(r'[ùúûü]', 'u', new)
    new= re.sub(r'[^\w\s]', " ", new)
    return new





def tratar(cadena, nlp, list_corregidas):
    #a,b = 'áéíóúü','aeiouu'
    #trans = str.maketrans(a,b)
    #elimino hiperenlaces
    tratada = re.sub(r'\s{0,}http+s{0,1}\S+\s{1,}', ' ', cadena)
    #elimino signos ortográficos
    tratada= re.sub(r'[^\w\s]', ' ', tratada)
    #quito los números
    tratada= re.sub('[0-9]', ' ', tratada)

    tratada = re.sub(r"\b[A-Z]{1}[a-z]+\b", " ", tratada)
    tratada = tratada.replace("\n", " ")




    list_inicial = tratada.split()

    list_final = [quitar_acentos(palabra)for palabra in list_inicial if palabra]


    if list_corregidas:

      cadena_tratada =   ' '.join(list_final)
      list_tratadas =  [token.lemma_ for token in nlp(cadena_tratada) if not token.is_stop and len(token.lemma_)>2 and token.lemma_ in list_corregidas ]


    else:
      list_final = [quitar_acentos(palabra) for palabra in list_inicial]
      cadena_tratada =   ' '.join(list_final)
      list_tratadas =  [token.lemma_ for token in nlp(cadena_tratada) if not token.is_stop and len(token.lemma_)>2 ]





    tratada = ' '.join(list_tratadas)



    return tratada



In [ ]:
df = pd.read_excel("df_limpia.xlsx")

In [ ]:
#leemos el conjunto de datos descargado mediante la API con registros a partir del 1 de Septiembre de 2023
df_test = pd.read_excel("/content/test.xlsx")
print(len(df_test))
print(df_test.columns)

#al igual que en el conjunto de datos train creamos un campo de fecha para facilitar los análisis
df_test["requested_datetime"] = df_test["requested_datetime"].apply(lambda x: x[0:10] )
df_test["fecha"] =  pd.to_datetime(df_test["requested_datetime"], format='%Y-%m-%d')

output= build_table(df_test[0:10], 'blue_light')
IPython.display.HTML(output)


1101
Index(['service_request_id', 'status', 'service_code', 'service_name', 'title',
       'description', 'requested_datetime', 'updated_datetime',
       'address_string', 'address_id', 'long', 'lat'],
      dtype='object')


service_request_id,status,service_code,service_name,title,description,requested_datetime,updated_datetime,address_string,address_id,long,lat,fecha
775031,closed,4849666,Policia Local,Calle acorazado potemkin,"En la calle acorazado potemkin entre la calle el tambor de hojalata y la calle El séptimo sello hay desde hace casi un año unas vallas de obra con el riesgo de caída y más con el aire, da la sensación como que es el delimitando algún tipo de caída de materiales de fachadas desde este invierno.\nAbandono total. Adjunto foto.",2023-09-01,2023-09-25T09:07:29Z,,,,,2023-09-01
775029,open,1403,PAVIMENTO RAICES,Vadorrey,"En vadorrey en el paseo de la rivera con calle Jesús Burriel Alias hay un árbol que está levantando toda la acera con las raíces además de ser muy frondoso con riesgo quizás de desplome.\nAdjunto foto marcando la zona,justo a un banco.",2023-09-01,2023-09-04T06:45:29Z,,,,,2023-09-01
775027,closed,1000012,Zaragoza Deporte,Pista multideporte Carlos Val en Valdespartera.,"Deberían de valorar después de tantos años existiendo la pista multideporte Carlos Val en el barrio de Valdespartera en la zona de calle Belle epoque,Volver a empezar y plaza Bambola,zona muy frecuentada y que en nada que cambie la hora no se puede hacer uso.\nLa contestación que recibo hace más de 5 años es que no hay partida presupuestaria,pero mi pregunta es: y no se podría hacer como en el potrero de Arcosur,barrio vecino,poner encendido por tiempo o algo?",2023-09-01,2023-09-05T06:06:35Z,,,,,2023-09-01
775026,closed,1000012,Zaragoza Deporte,Valdespartera,"En avenida Casablanca bajando desde puerta de la luz dirección fuente de la Junquera hay unas cuantas farolas en ambos sentidos fundidas y también en la calle Viridiana con calle la gran ilusión hay alguna farola fundida desde hace días.\nEn la manzana de calle belle epoque ,con volver a empezar con plaza bambola hay farolas fundidas.\nEn general en los barrios del sur hay muchas farolas fundidas o ocultas con árboles.\n\nTambién deberían de valorar después de tantos años existiendo la pista multideporte Carlos Val en el barrio de Valdespartera en la zona de calle Belle epoque,Volver a empezar y plaza Bambola,zona muy frecuentada y que en nada que cambie la hora no se puede hacer uso.\nLa contestación que recibo hace más de 5 años es que no hay partida presupuestaria,pero mi pregunta es: y no se podría hacer como en el potrero de Arcosur,barrio vecino,poner encendido por tiempo o algo?\nMatricula del punto de luz:",2023-09-01,2023-09-11T06:48:07Z,,,,,2023-09-01
775025,closed,5144579,Instituto Mpal. Salud Publica,Paloma herida,"Hola buenas hoy hemos llamado a la policia por un animal herido y nos han dicho que no estan para esas tonterías, hemos llamado al 112 y nos han dicho que la especie no era protegida y que no podian hacer nada.\nEl animal era una paloma y ha sufrido durante dos horas en el suelo agonizando, he intentado reanimarla y pedir ayuda y nadie ha querido venir por lo menos para ver como estaba e intentar tomar una decisión para que ningun animal sufra. Pero claro como es una paloma y esta tan mal juzgada su especie dejemos que sufra y agonice hasta la muerte la paloma sigue siendo un ser vivo y sufre igual que cualquier otro ser vivo del mundo. Se que hay muchas injusticias en el mundo pero si se puede ayudar.. aunque sea en que no sufra mas por lo menos se intenta y no se dan justificaciones sin importancia por ser una paloma. No es ninguna broma. Espero que algun dia haya un numero de teléfono para protección de las especies olvidadas de Aragon.\nMuchas gracias",2023-09-01,2023-09-05T10:04:22Z,,,,,2023-09-01
775019,open,9043968,Cultura,8 de septiembre /día internacional de la alfabetización,"Mi madre me imploro que le enseñase a leer siendo niño ,yo no sabía cómo y baje un día a pedir ideas a Los Titiriteros de Binéfar y Paco Paricio nos invitó a ver su ""Retablo de la Navidad"" ,era el año 2008 y les gustó la idea ..Tarde 12 años en pintar y crear el ""Abecedario Pedagógi

In [ ]:
'''
El objetivo de la predicción realizada sobre test es saber en qué medida los códigos de servicios
asignados manualmente coinciden con los códigos que vamos a predecir según el contenido de las reclamaciones. Es decir,
primero clasifico los service_code del conjunto test tal y como lo he hecho con el conjunto train (en base a una clasificación manual
de los mismos basada en las opciones ofrecidas por el Ayuntamiento en su página web) y luego compararlos con las predicciones
generados por un algoritmo
'''


#para reclasificar los servicios en test utilizamos los service_code del dataset train_reclasificado
#elijo todas sus posibles únicas combinaciones
clasificacion_servicios = df[['service_code', 'service_name', 'service_code_ant','service_name_ant']].drop_duplicates()
print(clasificacion_servicios.head)
#print(len(clasificacion_servicios))

#de esta lista para clasificar eliminamos aquellos service_code que corresponden a departamentos finales
#obteniendo el service_code, service_name y subcodigo (codigo sin clasificar)


#clasificacion_servicios.subcodigo = clasificacion_servicios.service_code_ant.fillna(0)
clasificacion_servicios.to_excel("clasificacion_servicios.xlsx")
'''
clasificacion_servicios.subcodigo = clasificacion_servicios[['service_code', 'subcodigo']].apply(lambda x: x.service_code if x.subcodigo == 0 else x.subcodigo, axis = 1 )
'''
#print("service_code posibles",len(clasificacion_servicios.subcodigo))

print(clasificacion_servicios.dtypes)
#clasificacion_servicios = pd.unique(df[['service_code', 'service_name', 'nuevo_codigo']])

<bound method NDFrame.head of        service_code                                       service_name  \
0           4849672                                   Limpieza Pública   
2           4849669                      Infraestructuras:Conservacion   
8           4849669                      Infraestructuras:Conservacion   
14          4849672                                   Limpieza Pública   
15          4849669                      Infraestructuras:Conservacion   
...             ...                                                ...   
21411       1000029  Oficina Jurídica de Servicios Públicos y Movil...   
23015       5144577                                  Alumbrado Publico   
24333       5144577                                  Alumbrado Publico   
27536       1000028                                           Igualdad   
33622       1000030                                         Tributaria   

       service_code_ant                                   service_name_ant  
0   

In [ ]:


#combino el conjunto test con los service_codes obtenidos por el campo service_code inicial

df_test = pd.merge(df_test, clasificacion_servicios[['service_code_ant', 'service_code']], how = "left", left_on = "service_code", right_on = "service_code_ant" )
print(len(df_test))

df_test.rename(columns={'service_code_y': 'service_code', 'service_name_y': 'service_name_ant' }, inplace = True)

#df_test.drop(['service_code'], axis = 1, inplace = True)

df_test = pd.merge(df_test, clasificacion_servicios[['service_code_ant', 'service_name']], how = "left", left_on = "service_code_x", right_on = "service_code_ant" )
df_test.to_excel("df_test.xlsx")


1101


In [ ]:

#el archivo listado_corregidas.xlsx se crea a partir de la visualización y edición manual de los registros en acorregir.xlsx
#elimino nombre propios, palabras como calle (que aparecen con mucha frecuencaa pero en principio pienso que no aportan mucho)...


#Lo que voy a hacer es tratar el conjunto test de la misma manera que he preprocesado el conjunto train


corregidas = pd.read_excel("/content/lista_corregidas.xlsx")
#print(corregidas.columns)
corregidas = corregidas[corregidas["sino"]==1]
list_corregidas = corregidas["token"].tolist()


nlp = spacy.load("es_core_news_lg")
lst_descripcion_test = df_test["description"].tolist()
texto_test = [tratar(doc,nlp, list_corregidas )  for doc in  lst_descripcion_test ]

df_test["texto_tratado"] = pd.Series(texto_test)



#localizo las filas que no tienen texto tokenizado en el campo texto_tratado y las elimino
print(len(df_test[df_test['texto_tratado'] == ''])  )
vacios_test = df_test[df_test['texto_tratado'] == '']
vacios_test.to_excel("vacios_test.xlsx")
print(len(df_test))
df_test.drop(df_test[df_test['texto_tratado'] == ''].index, inplace = True)
print(len(df_test))

texting_test = df_test["texto_tratado"].str.split()
print(texting_test.head)

#genero una lista de listas para su vectorización
corpus_test=[[word for word in elem] for elem in texting_test ]


#y el conjunto de palabras existentes en el campo descripción del dataset
corpux_test = [item for row in corpus_test for item in row if item]


#genero un dataframe con los resultados definitivos
df_test.to_excel("df_test_limpia.xlsx")
output= build_table(df_test.head(), 'blue_light')
IPython.display.HTML(output)


4
1101
1097
<bound method NDFrame.head of 0       [tambor, año, valla, obra, riesgo, caida, aire...
1       [paseo, arbol, levantar, acera, raiz, riesgo, ...
2       [deberiar, valorar, tanto, año, existir, pista...
3       [bajar, puerta, luz, direccion, fuente, farola...
4       [llamar, policia, animal, llamar, especie, pro...
                              ...                        
1096    [actividad, laboral, poligono, ciudadano, tran...
1097    [hoja, superar, marcado, ley, mañana, ruido, i...
1098                        [farola, fundido, punto, luz]
1099    [error, web, tabla, precio, instalación, depor...
1100    [marcha, interior, barrio, jardin, vertical, a...
Name: texto_tratado, Length: 1097, dtype: object>


service_request_id,status,service_code_x,service_name_x,title,description,requested_datetime,updated_datetime,address_string,address_id,long,lat,fecha,service_code_ant_x,service_code,service_code_ant_y,service_name_y,texto_tratado
775031,closed,4849666,Policia Local,Calle acorazado potemkin,"En la calle acorazado potemkin entre la calle el tambor de hojalata y la calle El séptimo sello hay desde hace casi un año unas vallas de obra con el riesgo de caída y más con el aire, da la sensación como que es el delimitando algún tipo de caída de materiales de fachadas desde este invierno.\nAbandono total. Adjunto foto.",2023-09-01,2023-09-25T09:07:29Z,,,,,2023-09-01,4849666,4849666,4849666,Policia Local,tambor año valla obra riesgo caida aire sensacion algun tipo caida material fachada invierno foto
775029,open,1403,PAVIMENTO RAICES,Vadorrey,"En vadorrey en el paseo de la rivera con calle Jesús Burriel Alias hay un árbol que está levantando toda la acera con las raíces además de ser muy frondoso con riesgo quizás de desplome.\nAdjunto foto marcando la zona,justo a un banco.",2023-09-01,2023-09-04T06:45:29Z,,,,,2023-09-01,1403,4849671,1403,Parques y Jardines,paseo arbol levantar acera raiz riesgo foto marcar justo banco
775027,closed,1000012,Zaragoza Deporte,Pista multideporte Carlos Val en Valdespartera.,"Deberían de valorar después de tantos años existiendo la pista multideporte Carlos Val en el barrio de Valdespartera en la zona de calle Belle epoque,Volver a empezar y plaza Bambola,zona muy frecuentada y que en nada que cambie la hora no se puede hacer uso.\nLa contestación que recibo hace más de 5 años es que no hay partida presupuestaria,pero mi pregunta es: y no se podría hacer como en el potrero de Arcosur,barrio vecino,poner encendido por tiempo o algo?",2023-09-01,2023-09-05T06:06:35Z,,,,,2023-09-01,1000012,1000012,1000012,Zaragoza Deporte,deberiar valorar tanto año existir pista barrio empezar plaza frecuentado cambiar hora contestacion recibir año pregunta potrero barrio vecino encendido tiempo
775026,closed,1000012,Zaragoza Deporte,Valdespartera,"En avenida Casablanca bajando desde puerta de la luz dirección fuente de la Junquera hay unas cuantas farolas en ambos sentidos fundidas y también en la calle Viridiana con calle la gran ilusión hay alguna farola fundida desde hace días.\nEn la manzana de calle belle epoque ,con volver a empezar con plaza bambola hay farolas fundidas.\nEn general en los barrios del sur hay muchas farolas fundidas o ocultas con árboles.\n\nTambién deberían de valorar después de tantos años existiendo la pista multideporte Carlos Val en el barrio de Valdespartera en la zona de calle Belle epoque,Volver a empezar y plaza Bambola,zona muy frecuentada y que en nada que cambie la hora no se puede hacer uso.\nLa contestación que recibo hace más de 5 años es que no hay partida presupuestaria,pero mi pregunta es: y no se podría hacer como en el potrero de Arcosur,barrio vecino,poner encendido por tiempo o algo?\nMatricula del punto de luz:",2023-09-01,2023-09-11T06:48:07Z,,,,,2023-09-01,1000012,1000012,1000012,Zaragoza Deporte,bajar puerta luz direccion fuente farola sentido fundida farola fundido manzana volver empezar plaza farola fundida general barrio sur farola fundida arbol deberiar valorar tanto año existir pista barrio empezar plaza frecuentado cambiar hora contestacion recibir año pregunta potrero barrio vecino encendido tiempo punto luz
775025,closed,5144579,Instituto Mpal. Salud Publica,Paloma herida,"Hola buenas hoy hemos llamado a la policia por un animal herido y nos han dicho que no estan para esas tonterías, hemos llamado al 112 y nos han dicho que la especie no era protegida y que no podian hacer nada.\nEl animal era una paloma y ha sufrido durante dos horas en el suelo agonizando, he intentado reanimarla y pedir ayuda y nadie ha querido venir por lo menos para ver como estaba e intentar tomar una decisión para que ningun animal sufra. Pero claro como es una paloma y esta tan mal juzgada s

In [ ]:


print(len(df_test))
texting = df["texto_tratado"].str.split()


#genero una lista de listas para su vectorización del cojunto train
corpus=[[word for word in elem] for elem in texting ]
print(corpus[0:10])
print(len(corpus))



1097
[['gente', 'venir', 'observar', 'excesivo', 'limpieza', 'centro', 'agua', 'llover', 'video', 'demostrar', 'necesario', 'agua', 'mayoria', 'gente', 'ciudad', 'ciudad', 'agua'], ['arreglar', 'echar', 'culpa', 'ciudadano', 'cargo', 'animal', 'sacar', 'necesidad', 'dejar', 'echar', 'verguenza', 'vecindario', 'palabra', 'limpieza', 'seguro', 'prestar', 'atencion', 'cercano', 'perro', 'llevar', 'semana', 'invitar', 'alcalde', 'pasear', 'disfrutar', 'acera', 'mobiliario', 'urbano', 'fachada', 'pis', 'continuo', 'perro', 'asqueroso', 'limpiar', 'fondo', 'multar', 'dueño', 'vecino', 'perro', 'tener', 'enviar', 'paseo'], ['bache', 'paseo', 'reparacion', 'bache'], ['bache', 'plaza', 'reparacion', 'bache'], ['bache', 'reparacion', 'bache'], ['bache', 'reparacion', 'bache'], ['unir', 'hecho', 'porqueria', 'entrada', 'peligroso', 'trabajador', 'empresa', 'hoja', 'mando', 'foto', 'hora'], ['limpiar', 'fondo', 'estar', 'plaza', 'alrededor'], ['malestar', 'vecino', 'lamentable', 'asfalto', 'grieta

In [ ]:


'''
Obtenemos vectores para cada uno de los documentos con TfidfVectorizer que es una herramienta de vectorización de textos
que nó solo tiene en cuenta el nñumero de veces que aparece una palabra en un documento, sino su
peso o frecuencia a lo largo de todo el conjunto de documentos o corpus
'''
#construimos el modelo
modelo_tfidf_sklearn = TfidfVectorizer(max_df=0.5, min_df=5, use_idf= False)

#generamos los vectores para el conjunto train
xtrain =modelo_tfidf_sklearn.fit_transform([' '.join(doc) for doc in corpus])

#generamos los vectores para el conjunto test
xtest = modelo_tfidf_sklearn.transform([' '.join(doc) for doc in corpus_test])


In [ ]:

#creamos un modelo de regresión logística
model = linear_model.LogisticRegression(solver='liblinear',max_iter=1000)

print(len(df))
print(xtrain.shape[0])
print(len(df.service_code))

#lo entrenamos sobre el campo service_code de xtrain
model.fit(xtrain, df.service_code)

#predecimos los service_code de test
preds = model.predict(xtest)
print(xtest.shape)


print(preds[0:10],df_test.service_code[0:10],df_test.service_request_id[0:10])

40513
40513
40513
(1097, 1692)
[4849669 4849671 1000012 5144577 5144579 4849671 4849671 1000000 4849671
 4849671] 0    4849666
1    4849671
2    1000012
3    1000012
4    5144579
5    9043968
6    9043968
7    5931009
8    4849671
9    4849671
Name: service_code, dtype: int64 0    775031
1    775029
2    775027
3    775026
4    775025
5    775019
6    775013
7    775010
8    775008
9    775006
Name: service_request_id, dtype: int64


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(preds, df_test.service_code))

#vemos que el f1-score, la media armónica de precision y recall, llega al 71%


              precision    recall  f1-score   support

     1000000       0.45      0.50      0.48        10
     1000002       0.38      1.00      0.55         3
     1000007       0.64      0.62      0.63        48
     1000008       0.92      1.00      0.96        12
     1000009       0.00      0.00      0.00         0
     1000010       0.33      0.33      0.33         3
     1000011       0.00      0.00      0.00         0
     1000012       0.38      0.50      0.43         6
     1000015       0.00      0.00      0.00         0
     1000017       0.17      0.50      0.25         2
     1000018       0.00      0.00      0.00         0
     1000028       0.00      0.00      0.00         0
     1000029       0.00      0.00      0.00         0
     1000030       0.12      1.00      0.22         1
     4784129       0.00      0.00      0.00         0
     4849665       0.00      0.00      0.00         0
     4849666       0.64      0.49      0.56       112
     4849668       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(df_testx.index)
print(type(preds))

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1099, 1100],
           dtype='int64', length=1097)
<class 'numpy.ndarray'>


In [ ]:

df_testx= df_test.copy()
df_testx.reset_index(inplace = True)
print(df_testx.head(10))
df_testx.rename(columns={'service_code': 'service_code_1','service_code_x': 'service_code_0', 'service_name_y': 'service_name_1', 'service_name_x': 'service_name_0' }, inplace = True)
predx = pd.DataFrame(data= preds, columns=["predict"])
#predx.reset_index(inplace =True)
print(predx.columns)
print(df_testx.columns)
#df_testx.reset_index(inplace = True)
print(len(predx))
print(len(df_testx))

df_test_pred = df_testx.join(predx, lsuffix='_left', rsuffix='_right',)

df_test_pred.to_excel("predicciones.xlsx")
print(df_test_pred.columns)



resultados = pd.merge(df_test_pred, clasificacion_servicios[['service_code', 'service_name']].drop_duplicates() ,how = 'left', left_on='predict', right_on='service_code')
print(len(resultados))
print(resultados.columns)





   index  service_request_id  status  service_code_x  \
0      0              775031  closed         4849666   
1      1              775029    open            1403   
2      2              775027  closed         1000012   
3      3              775026  closed         1000012   
4      4              775025  closed         5144579   
5      5              775019    open         9043968   
6      6              775013  closed         9043968   
7      7              775010  closed             310   
8      8              775008  closed            1303   
9      9              775006  closed             120   

                  service_name_x  \
0                  Policia Local   
1               PAVIMENTO RAICES   
2               Zaragoza Deporte   
3               Zaragoza Deporte   
4  Instituto Mpal. Salud Publica   
5                        Cultura   
6                        Cultura   
7    Filtración o salida de agua   
8                  PINZADO RAMAS   
9                     M

In [ ]:
resultados = resultados[['service_request_id','service_code_0','service_name_0', 'service_code_1','service_name_1','service_code_0','service_name_0','title', 'description', 'predict', 'service_name' ]]

resultados.to_excel("resultados.xlsx")

In [ ]:
#voy a ir probando con otros modelos de predicción a ver si podemos mejorar el F1

from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB().fit(xtrain, df.service_code)


predz = model.predict(xtest)


print(classification_report(predz, df_test.service_code))





              precision    recall  f1-score   support

     1000000       0.18      1.00      0.31         2
     1000002       0.12      1.00      0.22         1
     1000007       0.55      0.49      0.52        53
     1000008       0.85      1.00      0.92        11
     1000009       0.00      0.00      0.00         0
     1000010       0.67      0.50      0.57         4
     1000011       0.00      0.00      0.00         0
     1000012       0.25      1.00      0.40         2
     1000015       0.00      0.00      0.00         0
     1000017       0.00      0.00      0.00         0
     1000018       0.00      0.00      0.00         0
     1000028       0.00      0.00      0.00         0
     1000029       0.00      0.00      0.00         0
     1000030       0.00      0.00      0.00         0
     4784129       0.00      0.00      0.00         0
     4849665       0.00      0.00      0.00         0
     4849666       0.60      0.45      0.51       116
     4849668       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:

from sklearn.tree import DecisionTreeClassifier



model = DecisionTreeClassifier().fit(xtrain, df.service_code)


predz = model.predict(xtest)


print(classification_report(predz, df_test.service_code))




              precision    recall  f1-score   support

       40000       0.00      0.00      0.00         2
     1000000       0.00      0.00      0.00        14
     1000002       0.38      0.60      0.46         5
     1000007       0.53      0.47      0.50        53
     1000008       0.92      0.92      0.92        13
     1000009       0.00      0.00      0.00         1
     1000010       1.00      0.75      0.86         4
     1000011       0.00      0.00      0.00         3
     1000012       0.50      0.36      0.42        11
     1000014       0.00      0.00      0.00         3
     1000015       0.00      0.00      0.00         0
     1000016       0.00      0.00      0.00         4
     1000017       0.17      0.12      0.14         8
     1000018       0.00      0.00      0.00         2
     1000028       0.00      0.00      0.00         0
     1000029       0.00      0.00      0.00         4
     1000030       0.00      0.00      0.00         0
     4784129       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

En esta parte del código voy a estudiar si consigo mayor accuracy utilizando los datos en bruto de los datasets train y test sin reclasificar los service_code, utilizando la clasificación descargada de la API

In [ ]:
train  = pd.read_csv("/content/train.csv")
print(len(train))

#cambio el tipo de campo a string
train.service_code = train.service_code.astype('int64')

#añado campos de fecha para facilitar el análisis posterior
train["requested_datetime"] = train["requested_datetime"].apply(lambda x: x[0:10] )
train["fecha"] =  pd.to_datetime(train["requested_datetime"], format='%Y-%m-%d')
train["anyo"] = train['fecha'].dt.year

40594


In [ ]:
#aplico siempre el mismo preprocesamiento
corregidas = pd.read_excel("/content/lista_corregidas.xlsx")
print(corregidas.columns)
corregidas = corregidas[corregidas["sino"]==1]
list_corregidas = corregidas["token"].tolist()


nlp = spacy.load("es_core_news_lg")
lst_descripcion = train["description"].tolist()
texto = [tratar(doc,nlp, list_corregidas )  for doc in  lst_descripcion ]

train["texto_tratado"] = pd.Series(texto)

texting = train["texto_tratado"].str.split()
print(texting.head)

#genero una lista de listas para su vectorización
corpus=[[word for word in elem] for elem in texting ]

#y el conjunto de palabras existentes en el campo descripción del dataset
corpux = [item for row in corpus for item in row if item]

Index(['Unnamed: 0', 'token', 'ocurrencias', 'sino'], dtype='object')
<bound method NDFrame.head of 0        [gente, venir, observar, excesivo, limpieza, c...
1        [arreglar, echar, culpa, ciudadano, cargo, ani...
2        [mostrar, malestar, hecho, fiesta, sala, acaba...
3        [sugerir, proyecto, recogida, reciclaje, numer...
4        [documento, hecho, quedar, falta, seguridad, p...
                               ...                        
40589    [aspersor, ribera, altura, paseo, agua, tierra...
40590    [semaforo, peatón, direccion, justo, cruce, se...
40591    [barrendero, alrededor, foto, queja, mandar, q...
40592    [parque, menor, haber, riego, roto, aparecer, ...
40593    [parque, tormenta, mes, dejar, acera, cantidad...
Name: texto_tratado, Length: 40594, dtype: object>


In [ ]:
train.to_excel("train_bruto.xlsx")

In [ ]:
test = pd.read_excel("/content/test.xlsx")
print(len(test))

#cambio el tipo de campo a string
test.service_code = test.service_code.astype('int64')

#añado campos de fecha para facilitar el análisis posterior
test["requested_datetime"] = test["requested_datetime"].apply(lambda x: x[0:10] )
test["fecha"] =  pd.to_datetime(test["requested_datetime"], format='%Y-%m-%d')
test["anyo"] = test['fecha'].dt.year


1101


In [ ]:
#aplico siempre el mismo preprocesamiento
corregidas = pd.read_excel("/content/lista_corregidas.xlsx")
print(corregidas.columns)
corregidas = corregidas[corregidas["sino"]==1]
list_corregidas = corregidas["token"].tolist()


nlp = spacy.load("es_core_news_lg")
lst_descripcion = test["description"].tolist()
texto = [tratar(doc,nlp, list_corregidas )  for doc in  lst_descripcion ]

test["texto_tratado"] = pd.Series(texto)

texting = test["texto_tratado"].str.split()


#genero una lista de listas para su vectorización
corpus_test=[[word for word in elem] for elem in texting ]

#y el conjunto de palabras existentes en el campo descripción del dataset
corpux_test = [item for row in corpus_test for item in row if item]

test.to_excel("test_bruto.xlsx")

Index(['Unnamed: 0', 'token', 'ocurrencias', 'sino'], dtype='object')


In [ ]:
#vectorizo de nuevo y creo el modelo logistico

modelo_tfidf_sklearn = TfidfVectorizer(max_df=0.5, min_df=5, use_idf= True)
xtrain =modelo_tfidf_sklearn.fit_transform([' '.join(doc) for doc in corpus])

xtest = modelo_tfidf_sklearn.transform([' '.join(doc) for doc in corpus_test])

model = linear_model.LogisticRegression(solver='lbfgs',max_iter=500)

model.fit(xtrain, train.service_code)

preds = model.predict(xtest)


print(classification_report(preds, test.service_code))

              precision    recall  f1-score   support

          10       0.49      0.80      0.61        35
          20       0.50      0.11      0.18         9
          30       0.00      0.00      0.00         1
          50       0.50      0.50      0.50         2
          60       0.62      0.44      0.52        18
          70       0.00      0.00      0.00         0
          80       0.00      0.00      0.00         0
          85       0.00      0.00      0.00         1
          90       0.57      0.24      0.33        17
          91       0.50      0.50      0.50         2
          92       0.20      0.25      0.22         4
          95       0.00      0.00      0.00         0
          97       0.00      0.00      0.00         0
         110       0.50      0.50      0.50         2
         120       0.63      0.71      0.67        52
         150       0.25      0.20      0.22         5
         160       0.00      0.00      0.00         1
         170       0.60    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [ ]:
predx = pd.DataFrame(data= preds, columns=["predict"])



pred_merge = pd.merge(predx, df.drop_duplicates(subset=['service_code']), how= 'left', left_on='predict', right_on='service_code')
print(pred_merge.columns)

#elegimos las columnas del service_code predicho y su nombre
pred_merge = pred_merge[['service_code', 'service_name']]
#las renombramos para el merge posterior

pred_merge.rename(columns={'service_code': 'pred_service_code', 'service_name': 'pred_service_name', 'service_name_x' : 'service_name_ant' }, inplace = True)
print(pred_merge.head)
#verificamos tamaños
print(len(predx))
print(len(pred_merge))

#resultados = pred_merge.join(df_test, lsuffix='_left', rsuffix='_right')
resultados = pred_merge.join(df_test)

resultados = resultados[['service_request_id','service_code','service_name', 'pred_service_code', 'pred_service_name','title', 'description', 'service_code_ant', 'service_name_x' ]]

print(resultados.columns)



resultados.to_excel("resultados_bruto.xlsx")

Index(['predict', 'Unnamed: 0.2', 'Unnamed: 0.1', 'service_request_id',
       'status', 'service_code_ant', 'service_name_ant', 'title',
       'description', 'requested_datetime', 'updated_datetime',
       'address_string', 'address_id', 'long', 'lat', 'id_cat_sip', 'fecha',
       'anyo', 'Unnamed: 0', 'codigo', 'subcodigo', 'categoria_x',
       'subcategoria', 'service_name_y', 'service_code', 'service_name',
       'categoria', 'texto_tratado_inicial', 'texto_tratado'],
      dtype='object')
<bound method NDFrame.head of       pred_service_code                     pred_service_name
0             5144587.0                             Urbanismo
1                   NaN                                   NaN
2             1000012.0                      Zaragoza Deporte
3             5144577.0                     Alumbrado Publico
4             5144579.0         Instituto Mpal. Salud Publica
...                 ...                                   ...
1096         97550336.0  Movilid